In [1]:
import tensorflow as tf
import numpy as np
import time
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
from keras.datasets import mnist, cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()
from keras.utils import np_utils

row, col = img_train.shape[1], img_train.shape[2]

if len(img_train.shape) == 4 :
    depth = img_train.shape[3]
else:
    depth = 1
    img_train = img_train.reshape(img_train.shape[0], row, col, depth)
    img_test = img_test.reshape(img_test.shape[0], row, col, depth)

img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

img_train = img_train - np.mean(img_train, axis=0)
img_test = img_test - np.mean(img_test, axis= 0)

img_train = img_train / np.var(img_train, axis=0)
img_test = img_test / np.var(img_test, axis =0)

label_train = np_utils.to_categorical(label_train, 10)
label_test = np_utils.to_categorical(label_test, 10)


Using TensorFlow backend.


In [3]:
INIT_SIZE = 4
BATCH_SIZE = 256
LEARNING_RATE = 0.001
EPOCHS = 1000

In [4]:
X = tf.placeholder(tf.float32, [None, row, col, depth])
Y = tf.placeholder(tf.float32, [None, 10])

with tf.variable_scope('conv1') as scope:
    conv_W1 = tf.get_variable("W", shape=[3, 3, 3, INIT_SIZE ], 
                         initializer=tf.contrib.layers.xavier_initializer())
    conv_L1 = tf.nn.conv2d(X, conv_W1, strides=[1, 1, 1, 1], 
                           padding='SAME', name='conv')
    conv_L1_BN = tf.layers.batch_normalization(conv_L1, axis=-1, name='BN')
    conv_L1_act = tf.nn.relu(conv_L1_BN, name='relu')

with tf.variable_scope('conv2') as scope:
    conv_W2 = tf.get_variable("W", shape=[3, 3, INIT_SIZE, INIT_SIZE ], 
                         initializer=tf.contrib.layers.xavier_initializer())
    conv_L2 = tf.nn.conv2d(conv_L1_act, conv_W2, strides=[1, 1, 1, 1], 
                           padding='SAME', name='conv')
    conv_L2_BN = tf.layers.batch_normalization(conv_L2, axis=-1, name='BN')
    conv_L2_act = tf.nn.relu(conv_L2_BN, name='relu')
    
    pool1 = tf.nn.max_pool(conv_L2_act, ksize=[1, 2, 2, 1],
                          strides = [1, 2, 2, 1], padding='SAME',
                          name='pool')

with tf.variable_scope('conv3') as scope:
    conv_W3 = tf.get_variable("W", shape=[3, 3, INIT_SIZE, INIT_SIZE*2 ], 
                         initializer=tf.contrib.layers.xavier_initializer())
    conv_L3 = tf.nn.conv2d(pool1, conv_W3, strides=[1, 1, 1, 1], 
                           padding='SAME', name='conv')
    conv_L3_BN = tf.layers.batch_normalization(conv_L3, axis=-1, name='BN')
    conv_L3_act = tf.nn.relu(conv_L3_BN, name='relu')

with tf.variable_scope('conv4') as scope:
    conv_W4 = tf.get_variable("W", shape=[3, 3, INIT_SIZE*2, INIT_SIZE*2],
                             initializer=tf.contrib.layers.xavier_initializer())
    conv_L4 = tf.nn.conv2d(conv_L3_act, conv_W4, strides=[1, 1, 1, 1],
                          padding='SAME', name='conv')
    conv_L4_BN = tf.layers.batch_normalization(conv_L4, axis=-1, name='BN')
    conv_L4_act = tf.nn.relu(conv_L4_BN, name='relu')
    
    pool2 = tf.nn.max_pool(conv_L4_act, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME',
                          name='pool')

with tf.variable_scope('conv5') as scope:
    conv_W5 = tf.get_variable("W", 
                              shape=[3, 3, INIT_SIZE*2, INIT_SIZE*4],
                              initializer=tf.contrib.layers.xavier_initializer())
    conv_L5 = tf.nn.conv2d(pool2, conv_W5, strides=[1, 1, 1, 1],
                          padding='SAME', name='conv')
    conv_L5_BN = tf.layers.batch_normalization(conv_L5, axis=-1, name='BN')
    conv_L5_act = tf.nn.relu(conv_L5_BN, name='relu')
    
with tf.variable_scope('conv6') as scope:
    conv_W6 = tf.get_variable("W", 
                              shape=[3, 3, INIT_SIZE*4, INIT_SIZE*4],
                              initializer=tf.contrib.layers.xavier_initializer())
    conv_L6 = tf.nn.conv2d(conv_L5_act, conv_W6, strides=[1, 1, 1, 1],
                          padding='SAME', name='conv')
    conv_L6_BN = tf.layers.batch_normalization(conv_L6, axis=-1, name='BN')
    conv_L6_act = tf.nn.relu(conv_L6_BN, name='relu')
    
with tf.variable_scope('conv7') as scope:
    conv_W7 = tf.get_variable("W", 
                              shape=[3, 3, INIT_SIZE*4, INIT_SIZE*4],
                              initializer=tf.contrib.layers.xavier_initializer())
    conv_L7 = tf.nn.conv2d(conv_L6_act, conv_W7, strides=[1, 1, 1, 1],
                          padding='SAME', name='conv')
    conv_L7_BN = tf.layers.batch_normalization(conv_L7, axis=-1, name='BN')
    conv_L7_act = tf.nn.relu(conv_L7_BN, name='relu')
    pool3 = tf.nn.max_pool(conv_L7_act, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME',
                          name='pool')

with tf.variable_scope('flat') as scope:
    flat_shape = 4*4*INIT_SIZE*4
    flat = tf.reshape(pool3, [-1, flat_shape], name='flat')
    
with tf.variable_scope('fc1') as scope:
    fc_W1 = tf.get_variable("W", shape=[flat_shape, INIT_SIZE*8],
                           initializer=tf.contrib.layers.xavier_initializer())
    fc_b1 = tf.Variable(tf.random_normal([INIT_SIZE*8]), name='b')
    fc_L1 = tf.add(tf.matmul(flat, fc_W1), fc_b1, name='dense')
    fc_L1_BN = tf.layers.batch_normalization(fc_L1, axis=-1, name='BN')
    fc_L1_act = tf.nn.relu(fc_L1_BN, name='relu')
    
with tf.variable_scope('fc2') as scope:
    fc_W2 = tf.get_variable("W", shape=[INIT_SIZE*8, 10],
                           initializer=tf.contrib.layers.xavier_initializer())
    fc_b2 = tf.Variable(tf.random_normal([10]), name='b')
    logits = tf.add(tf.matmul(fc_L1_act, fc_W2), fc_b2, name='dense')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
tf.trainable_variables()

[<tf.Variable 'conv1/W:0' shape=(3, 3, 3, 4) dtype=float32_ref>,
 <tf.Variable 'conv1/BN/beta:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv1/BN/gamma:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv2/W:0' shape=(3, 3, 4, 4) dtype=float32_ref>,
 <tf.Variable 'conv2/BN/beta:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv2/BN/gamma:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv3/W:0' shape=(3, 3, 4, 8) dtype=float32_ref>,
 <tf.Variable 'conv3/BN/beta:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'conv3/BN/gamma:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'conv4/W:0' shape=(3, 3, 8, 8) dtype=float32_ref>,
 <tf.Variable 'conv4/BN/beta:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'conv4/BN/gamma:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'conv5/W:0' shape=(3, 3, 8, 16) dtype=float32_ref>,
 <tf.Variable 'conv5/BN/beta:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv5/BN/gamma:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv6/W:0' shape=(3, 3

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print ('Learning Started.')
for epoch in range(EPOCHS):
    start = time.time()
    avg_cost = 0
    total_batch = int(img_train.shape[0] / BATCH_SIZE)
    
    for i in range(total_batch):
        idxs = np.random.permutation(xrange(len(img_train)))
        idxs_i = idxs[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print '=================================='
    print 'Epoch: {:04d}'.format(epoch + 1), 'cost = {:.6f}'.format(avg_cost)
    print 'Train Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_train, Y: label_train}))
    print 'Test Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_test, Y: label_test}))
    end = time.time()
    print 'Elapsed time: {:.6f} sec'.format(end-start)

Learning Started.
Epoch: 0001 cost = 2.024865
Train Accuracy: 0.377020
Test Accuracy: 0.376900
Elapsed time: 12.882826 sec
Epoch: 0002 cost = 1.622774
Train Accuracy: 0.420580
Test Accuracy: 0.413200
Elapsed time: 8.417429 sec
Epoch: 0003 cost = 1.510774
Train Accuracy: 0.464500
Test Accuracy: 0.457100
Elapsed time: 7.024525 sec
Epoch: 0004 cost = 1.433419


KeyboardInterrupt: 

In [43]:
tf.gradients

<function tensorflow.python.ops.gradients_impl.gradients>

In [21]:
weight = sess.run({v.name: v for v in tf.trainable_variables()})

In [42]:
weight['fc2/b:0']

array([ 0.67440271,  0.53153807,  0.33823761, -1.09823704,  1.21694469,
       -0.82469535,  0.95711958,  1.5726614 , -0.98721391, -0.96824324], dtype=float32)

In [38]:
weight[a[]].shape

(3, 3, 8, 8)

In [34]:
list(weight.keys())

[u'conv6/BN/beta:0',
 u'conv7/BN/gamma:0',
 u'conv2/W:0',
 u'conv2/BN/gamma:0',
 u'conv4/W:0',
 u'conv2/BN/beta:0',
 u'conv1/BN/beta:0',
 u'fc1/BN/gamma:0',
 u'conv5/BN/beta:0',
 u'conv7/BN/beta:0',
 u'conv5/BN/gamma:0',
 u'conv3/W:0',
 u'conv1/BN/gamma:0',
 u'fc2/W:0',
 u'conv7/W:0',
 u'fc1/W:0',
 u'fc1/BN/beta:0',
 u'conv5/W:0',
 u'conv3/BN/beta:0',
 u'fc1/b:0',
 u'conv1/W:0',
 u'conv3/BN/gamma:0',
 u'conv6/BN/gamma:0',
 u'conv4/BN/beta:0',
 u'fc2/b:0',
 u'conv4/BN/gamma:0',
 u'conv6/W:0']

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES']='1'

print ('Learning Started.')
for epoch in range(epochs):
    start = time.time()
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size)
    
    for i in range(total_batch):
        idxs = np.random.permutation(xrange(len(img_train)))
        idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print '=================================='
    print 'Epoch: {:04d}'.format(epoch + 1), 'cost = {:.6f}'.format(avg_cost)
    print 'Train Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_train, Y: label_train}))
    print 'Test Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_test, Y: label_test}))
    end = time.time()
    print 'Elapsed time: {:.6f} sec'.format(end-start)

Learning Started.
Epoch: 0001 cost = 593.967533
Train Accuracy: 0.207820
Test Accuracy: 0.210500
Elapsed time: 5.121425 sec
Epoch: 0002 cost = 513.462055
Train Accuracy: 0.325360
Test Accuracy: 0.336900
Elapsed time: 4.775901 sec
Epoch: 0003 cost = 466.275428
Train Accuracy: 0.366880
Test Accuracy: 0.374100
Elapsed time: 5.069203 sec
Epoch: 0004 cost = 442.234666
Train Accuracy: 0.398860
Test Accuracy: 0.394500
Elapsed time: 4.940844 sec
Epoch: 0005 cost = 423.254033
Train Accuracy: 0.418560
Test Accuracy: 0.417500
Elapsed time: 4.759971 sec
Epoch: 0006 cost = 414.042198
Train Accuracy: 0.434960
Test Accuracy: 0.436400
Elapsed time: 4.999109 sec
Epoch: 0007 cost = 405.156913
Train Accuracy: 0.441260
Test Accuracy: 0.435600
Elapsed time: 4.864848 sec
Epoch: 0008 cost = 399.031918
Train Accuracy: 0.453700
Test Accuracy: 0.451900
Elapsed time: 5.011470 sec
Epoch: 0009 cost = 394.610902
Train Accuracy: 0.456100
Test Accuracy: 0.453800
Elapsed time: 5.059354 sec
Epoch: 0010 cost = 388.74221

In [ ]:
print ('Accuracy:', sess.run(accuracy, feed_dict={X: img_test, Y: label_test}))

In [24]:

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
#X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size)

    for i in range(total_batch):
        idxs = np.random.permutation(range(len(img_train)))
        idxs_i = idxs[i * batch_size: (i + 1) * batch_size]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
      X: img_test, Y: label_test}))
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy


Learning started. It takes sometime.
('Accuracy:', 0.96750015)
('Epoch:', '0001', 'cost =', '0.317009241')
('Accuracy:', 0.97760016)
('Epoch:', '0002', 'cost =', '0.090142726')
('Accuracy:', 0.9825002)
('Epoch:', '0003', 'cost =', '0.070426149')
('Accuracy:', 0.98270005)
('Epoch:', '0004', 'cost =', '0.056876211')


KeyboardInterrupt: 